In [1]:
import tensorflow as tf
from PIL import Image
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator , img_to_array, load_img
from keras.optimizers import RMSprop
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from glob import glob
from keras.models import Model
from keras.layers import Flatten, Dense
from sklearn.metrics import classification_report, confusion_matrix
from keras.applications import ResNet50V2, MobileNetV2, InceptionV3


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#!pip install rembg[gpu]
from rembg import remove
output = ["BGR","ORG","Crop"]

In [4]:
# Changable parameters
mode = "Crop"
IMAGE_HEIGHT_WIDTH = 350

In [5]:
if (mode == "BGR"):
#Training set background removal

  directory1 ='/content/drive/MyDrive/Projects/GP Project/Dataset/TrainSet'
  new_created1 = '/content/drive/MyDrive/Projects/GP Project/Dataset/BgRemoved/Training'

  for filename in glob(f'{directory1}/*'):
    folder = os.path.basename(filename)
    try:
      path1 = os.path.join(new_created1, folder)
      os.makedirs(path1)  
      for image in glob(f'{filename}/*'):
          image_extend = os.path.basename(image)
          input_path = image
          output_path = os.path.join(path1, image_extend)
          input = Image.open(input_path)
          output = remove(input)
          output.save(output_path)
    except FileExistsError:
      # directory already exists
      pass

#validition set background removal
  directory2 ='/content/drive/MyDrive/Projects/GP Project/Dataset/ValidationSet'
  new_created2 = '/content/drive/MyDrive/Projects/GP Project/Dataset/BgRemoved/Validation'

  for filename in glob(f'{directory2}/*'):
    folder = os.path.basename(filename)
    try:
      path2 = os.path.join(new_created2, folder)
      os.makedirs(path2)  
      for image in glob(f'{filename}/*'):
          image_extend = os.path.basename(image)
          input_path = image
          output_path = os.path.join(path2, image_extend)
          input = Image.open(input_path)
          output = remove(input)
          output.save(output_path)
    except FileExistsError:
      # directory already exists
      pass

#validition set background removal
  directory3 ='/content/drive/MyDrive/Projects/GP Project/Dataset/TestSet'
  new_created3 = '/content/drive/MyDrive/Projects/GP Project/Dataset/BgRemoved/Testing'

  try:  
    os.makedirs(new_created3)  
    for image in glob("/content/drive/MyDrive/Projects/GP Project/Dataset/TestSet/*.png"):
        image_extend = os.path.basename(image)
        input_path = image
        output_path = os.path.join(new_created3, image_extend)
        input = Image.open(input_path)
        output = remove(input)
        output.save(output_path)
  except FileExistsError:
    # directory already exists
    pass

else: pass



In [6]:
train = ImageDataGenerator(rescale = 1./255)
validation = ImageDataGenerator(rescale = 1./255)
#train = ImageDataGenerator()
#validation = ImageDataGenerator()
if (mode == "ORG"):
  #training and validation with the original dataset
  train_dataset = train.flow_from_directory('/content/drive/MyDrive/Projects/GP Project/Dataset/TrainSet',
                                          target_size = (IMAGE_HEIGHT_WIDTH,IMAGE_HEIGHT_WIDTH),
                                          batch_size = 32,
                                          class_mode = "categorical")
  validation_dataset = validation.flow_from_directory('/content/drive/MyDrive/Projects/GP Project/Dataset/ValidationSet',
                                                    target_size = (IMAGE_HEIGHT_WIDTH,IMAGE_HEIGHT_WIDTH),
                                                    batch_size = 32,
                                                    class_mode = "categorical")
elif mode == "Crop":
    #training and validation with the cropped dataset
  train_dataset = train.flow_from_directory('/content/drive/MyDrive/Projects/GP Project/Dataset/croppedDataset/trainSet',
                                          target_size = (IMAGE_HEIGHT_WIDTH,IMAGE_HEIGHT_WIDTH),
                                          batch_size = 25,
                                          class_mode = "categorical")
  validation_dataset = validation.flow_from_directory('/content/drive/MyDrive/Projects/GP Project/Dataset/croppedDataset/validSet',
                                                    target_size = (IMAGE_HEIGHT_WIDTH,IMAGE_HEIGHT_WIDTH),
                                                    batch_size = 25,
                                                    class_mode = "categorical")
else:
  #training and validation with the background removal dataset
  train_dataset = train.flow_from_directory('/content/drive/MyDrive/Projects/GP Project/Dataset/BgRemoved/Training',
                                          target_size = (IMAGE_HEIGHT_WIDTH,IMAGE_HEIGHT_WIDTH),
                                          batch_size = 32,
                                          class_mode = "categorical")
  validation_dataset = validation.flow_from_directory('/content/drive/MyDrive/Projects/GP Project/Dataset/BgRemoved/Validation',
                                                    target_size = (IMAGE_HEIGHT_WIDTH,IMAGE_HEIGHT_WIDTH),
                                                    batch_size = 32,
                                                    class_mode = "categorical")

Found 140 images belonging to 7 classes.
Found 31 images belonging to 7 classes.


In [7]:
train_dataset.class_indices
my_dic = dict(train_dataset.class_indices)
my_dic

{'bed': 0,
 'father': 1,
 'full': 2,
 'police': 3,
 'shirt': 4,
 'water': 5,
 'wrong': 6}

In [8]:
if mode == "Crop":
  # useful for getting number of classes
  folders = glob('/content/drive/MyDrive/Projects/GP Project/Dataset/croppedDataset/validSet/*')
elif mode == "ORG":
  folders = glob('/content/drive/MyDrive/Projects/GP Project/Dataset/ValidationSet/*')
else:
  folders = glob('/content/drive/MyDrive/Projects/GP Project/Dataset/BgRemoved/Validation/*')

# Model 1 VGG16

In [9]:
vgg = VGG16(input_shape=(IMAGE_HEIGHT_WIDTH,IMAGE_HEIGHT_WIDTH,3), weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False
  
# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 350, 350, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 350, 350, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 350, 350, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 175, 175, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 175, 175, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 175, 175, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 87, 87, 128)       0     

In [10]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit_generator(steps_per_epoch=len(train_dataset), generator=train_dataset, validation_data=validation_dataset, validation_steps=len(validation_dataset), epochs=6, callbacks=[checkpoint,early])

<ipython-input-10-bca8a532f54f>:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(steps_per_epoch=len(train_dataset), generator=train_dataset, validation_data=validation_dataset, validation_steps=len(validation_dataset), epochs=6, callbacks=[checkpoint,early])


Epoch 1/6
6/6 [==============================] - ETA: 0s - loss: 1.7329 - accuracy: 0.4571
Epoch 1: val_accuracy improved from -inf to 0.74194, saving model to vgg16_1.h5
6/6 [==============================] - 17s 2s/step - loss: 1.7329 - accuracy: 0.4571 - val_loss: 0.9218 - val_accuracy: 0.7419
Epoch 2/6
6/6 [==============================] - ETA: 0s - loss: 0.3517 - accuracy: 0.8714
Epoch 2: val_accuracy improved from 0.74194 to 0.77419, saving model to vgg16_1.h5
6/6 [==============================] - 3s 452ms/step - loss: 0.3517 - accuracy: 0.8714 - val_loss: 0.7138 - val_accuracy: 0.7742
Epoch 3/6
6/6 [==============================] - ETA: 0s - loss: 0.0540 - accuracy: 0.9929
Epoch 3: val_accuracy did not improve from 0.77419
6/6 [==============================] - 2s 443ms/step - loss: 0.0540 - accuracy: 0.9929 - val_loss: 0.6673 - val_accuracy: 0.7419
Epoch 4/6
6/6 [==============================] - ETA: 0s - loss: 0.0216 - accuracy: 0.9929
Epoch 4: val_accuracy did not improve

# Model 2 from Related work

## Model 2.1 "It was said that this model was overfitting"

In [11]:
model2_1 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=10, 
                         kernel_size=3, # can also be (3, 3)
                         activation="relu", 
                         input_shape=(IMAGE_HEIGHT_WIDTH, IMAGE_HEIGHT_WIDTH, 3)), # first layer specifies input shape (height, width, colour channels)
  tf.keras.layers.MaxPool2D(pool_size=2, # pool_size can also be (2, 2)
                            padding="valid"), # padding can also be 'same'
  tf.keras.layers.Conv2D(filters=10, 
                         kernel_size=3, 
                         activation="relu"
                         ),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(len(folders), activation="softmax") # binary activation output
])
model2_1.summary()

# Compile the model
model2_1.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 348, 348, 10)      280       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 174, 174, 10)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 172, 172, 10)      910       
                                                                 
 flatten_1 (Flatten)         (None, 295840)            0         
                                                                 
 dense_1 (Dense)             (None, 7)                 2070887   
                                                                 
Total params: 2,072,077
Trainable params: 2,072,077
Non-trainable params: 0
______________________________________________

In [12]:
history_1 = model2_1.fit(train_dataset,
                        epochs=20,
                        steps_per_epoch=len(train_dataset),
                        validation_data=validation_dataset,
                        validation_steps=len(validation_dataset))

Epoch 1/20
6/6 [==============================] - 2s 242ms/step - loss: 21.0278 - accuracy: 0.1786 - val_loss: 7.8192 - val_accuracy: 0.2258
Epoch 2/20
6/6 [==============================] - 1s 152ms/step - loss: 5.4481 - accuracy: 0.3429 - val_loss: 2.5965 - val_accuracy: 0.4516
Epoch 3/20
6/6 [==============================] - 1s 149ms/step - loss: 1.4692 - accuracy: 0.6214 - val_loss: 1.7125 - val_accuracy: 0.3226
Epoch 4/20
6/6 [==============================] - 1s 149ms/step - loss: 1.0056 - accuracy: 0.7929 - val_loss: 1.8583 - val_accuracy: 0.2581
Epoch 5/20
6/6 [==============================] - 1s 147ms/step - loss: 0.7537 - accuracy: 0.8714 - val_loss: 1.4549 - val_accuracy: 0.4516
Epoch 6/20
6/6 [==============================] - 1s 146ms/step - loss: 0.3956 - accuracy: 0.9857 - val_loss: 1.3280 - val_accuracy: 0.5484
Epoch 7/20
6/6 [==============================] - 1s 153ms/step - loss: 0.1846 - accuracy: 0.9857 - val_loss: 1.2895 - val_accuracy: 0.5484
Epoch 8/20
6/6 [===

In [13]:
model2_1.save('Model2.1_Related_cropped.h5')

## Model 2.2

In [14]:
model2_2 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=32, 
                         kernel_size=3, 
                         activation="relu", 
                         input_shape=(IMAGE_HEIGHT_WIDTH, IMAGE_HEIGHT_WIDTH, 3)), 
  tf.keras.layers.Conv2D(64, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2, # pool_size can also be (2, 2)
                            padding="valid"), # padding can also be 'same'
  tf.keras.layers.Conv2D(128, 3, activation="relu"),
  tf.keras.layers.Conv2D(32, 3, activation="relu"), # activation='relu' == tf.keras.layers.Activations(tf.nn.relu)
  tf.keras.layers.MaxPool2D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(len(folders), activation="softmax") # binary activation output
])

model2_2.summary()
# Compile the model
model2_2.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 348, 348, 32)      896       
                                                                 
 conv2d_3 (Conv2D)           (None, 346, 346, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 173, 173, 64)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 171, 171, 128)     73856     
                                                                 
 conv2d_5 (Conv2D)           (None, 169, 169, 32)      36896     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 84, 84, 32)       0         
 2D)                                                  

In [15]:
# Fit the model
history_2 = model2_2.fit(train_dataset,
                        epochs=40,
                        steps_per_epoch=len(train_dataset),
                        validation_data=validation_dataset,
                        validation_steps=len(validation_dataset))

Epoch 1/40
6/6 [==============================] - 9s 975ms/step - loss: 2.2360 - accuracy: 0.1714 - val_loss: 1.7810 - val_accuracy: 0.1613
Epoch 2/40
6/6 [==============================] - 2s 352ms/step - loss: 1.2549 - accuracy: 0.5571 - val_loss: 1.8279 - val_accuracy: 0.4194
Epoch 3/40
6/6 [==============================] - 2s 318ms/step - loss: 0.5755 - accuracy: 0.8286 - val_loss: 2.2981 - val_accuracy: 0.4839
Epoch 4/40
6/6 [==============================] - 2s 337ms/step - loss: 0.0900 - accuracy: 0.9714 - val_loss: 3.5760 - val_accuracy: 0.5484
Epoch 5/40
6/6 [==============================] - 2s 310ms/step - loss: 0.0147 - accuracy: 1.0000 - val_loss: 5.3509 - val_accuracy: 0.5161
Epoch 6/40
6/6 [==============================] - 2s 314ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 6.7729 - val_accuracy: 0.4516
Epoch 7/40
6/6 [==============================] - 2s 335ms/step - loss: 9.3866e-05 - accuracy: 1.0000 - val_loss: 8.0423 - val_accuracy: 0.4516
Epoch 8/40
6/6 [

In [16]:
model2_2.save('Model2.2_Related_cropped.h5')

# Model 3 ResNetV2 50,101,152 

- 50 all output was cow
- 101 most of the output was cow
- 152 take 55 minutes to train and 10 minutes to predict  

In [17]:
res = ResNet50V2(include_top=False, weights="imagenet",input_shape=(IMAGE_HEIGHT_WIDTH,IMAGE_HEIGHT_WIDTH,3))

for layer in res.layers:
  layer.trainable = False  

# our layers - you can add more if you want
x = Flatten()(res.output)
predictions = Dense(len(folders), activation='softmax')(x)

# create a model object
model3 = Model(inputs = res.input, outputs = predictions)

# view the structure of the model
model3.summary()

# tell the model what cost and optimization method to use
model3.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 350, 350, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 356, 356, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 175, 175, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [18]:
hist2 = model3.fit_generator(steps_per_epoch=len(train_dataset), generator=train_dataset, validation_data=validation_dataset, validation_steps=len(validation_dataset), epochs=6)


<ipython-input-18-12a0c716d8ce>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist2 = model3.fit_generator(steps_per_epoch=len(train_dataset), generator=train_dataset, validation_data=validation_dataset, validation_steps=len(validation_dataset), epochs=6)


Epoch 1/6
6/6 [==============================] - 10s 904ms/step - loss: 7.2303 - accuracy: 0.6071 - val_loss: 11.4206 - val_accuracy: 0.7097
Epoch 2/6
6/6 [==============================] - 1s 238ms/step - loss: 2.4199 - accuracy: 0.9071 - val_loss: 9.9261 - val_accuracy: 0.7097
Epoch 3/6
6/6 [==============================] - 2s 264ms/step - loss: 0.5672 - accuracy: 0.9714 - val_loss: 9.9651 - val_accuracy: 0.8065
Epoch 4/6
6/6 [==============================] - 2s 238ms/step - loss: 8.2080e-06 - accuracy: 1.0000 - val_loss: 8.4780 - val_accuracy: 0.8065
Epoch 5/6
6/6 [==============================] - 1s 239ms/step - loss: 3.4060e-09 - accuracy: 1.0000 - val_loss: 8.3904 - val_accuracy: 0.7742
Epoch 6/6
6/6 [==============================] - 1s 238ms/step - loss: 2.5545e-09 - accuracy: 1.0000 - val_loss: 8.6631 - val_accuracy: 0.7742


In [19]:
model3.save('res50V2_cropped.h5')

# Model 4 MobileNet

In [20]:
mobile = MobileNetV2(input_shape=(IMAGE_HEIGHT_WIDTH,IMAGE_HEIGHT_WIDTH,3), weights='imagenet', include_top=False)

# don't train existing weights
for layer in mobile.layers:
  layer.trainable = False

# our layers - you can add more if you want
x = Flatten()(mobile.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model4 = Model(inputs=mobile.input, outputs=prediction)

# view the structure of the model
model4.summary()

# tell the model what cost and optimization method to use
model4.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 350, 350, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 175, 175, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 175, 175, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [21]:
hist2 = model4.fit_generator(steps_per_epoch=len(train_dataset), generator=train_dataset, validation_data=validation_dataset, validation_steps=len(validation_dataset), epochs=6)


<ipython-input-21-07a7a248712f>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist2 = model4.fit_generator(steps_per_epoch=len(train_dataset), generator=train_dataset, validation_data=validation_dataset, validation_steps=len(validation_dataset), epochs=6)


Epoch 1/6
6/6 [==============================] - 6s 514ms/step - loss: 4.6150 - accuracy: 0.5857 - val_loss: 2.6548 - val_accuracy: 0.7419
Epoch 2/6
6/6 [==============================] - 1s 160ms/step - loss: 1.0702 - accuracy: 0.9214 - val_loss: 2.8351 - val_accuracy: 0.7097
Epoch 3/6
6/6 [==============================] - 1s 160ms/step - loss: 0.0144 - accuracy: 0.9929 - val_loss: 3.4954 - val_accuracy: 0.8065
Epoch 4/6
6/6 [==============================] - 1s 163ms/step - loss: 0.1387 - accuracy: 0.9786 - val_loss: 2.5921 - val_accuracy: 0.8065
Epoch 5/6
6/6 [==============================] - 1s 166ms/step - loss: 0.0415 - accuracy: 0.9929 - val_loss: 2.0774 - val_accuracy: 0.8065
Epoch 6/6
6/6 [==============================] - 1s 167ms/step - loss: 1.5326e-06 - accuracy: 1.0000 - val_loss: 2.3236 - val_accuracy: 0.8065


In [22]:
model4.save('mobileV2_cropped.h5')

# Model 5 InceptionV3

In [23]:
incept = InceptionV3(input_shape=(IMAGE_HEIGHT_WIDTH,IMAGE_HEIGHT_WIDTH,3), weights='imagenet', include_top=False)

# don't train existing weights
for layer in incept.layers:
  layer.trainable = False

# our layers - you can add more if you want
x = Flatten()(incept.output)

prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model5 = Model(inputs=incept.input, outputs=prediction)

# view the structure of the model
model5.summary()

# tell the model what cost and optimization method to use
model5.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

87910968/87910968 [==============================] - 5s 0us/step
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 350, 350, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_6 (Conv2D)              (None, 174, 174, 32  864         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 174, 174, 32  96         ['conv2d_6[0][0]']               
 alization)                

In [24]:
hist3 = model5.fit_generator(steps_per_epoch=len(train_dataset), generator=train_dataset, validation_data=validation_dataset, validation_steps=len(validation_dataset), epochs=6)


<ipython-input-24-5bb4ef50ac0b>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist3 = model5.fit_generator(steps_per_epoch=len(train_dataset), generator=train_dataset, validation_data=validation_dataset, validation_steps=len(validation_dataset), epochs=6)


Epoch 1/6
6/6 [==============================] - 12s 1s/step - loss: 14.1969 - accuracy: 0.3429 - val_loss: 13.5750 - val_accuracy: 0.5484
Epoch 2/6
6/6 [==============================] - 1s 206ms/step - loss: 4.2708 - accuracy: 0.7929 - val_loss: 8.8206 - val_accuracy: 0.5484
Epoch 3/6
6/6 [==============================] - 1s 201ms/step - loss: 2.5763 - accuracy: 0.8500 - val_loss: 4.5805 - val_accuracy: 0.6774
Epoch 4/6
6/6 [==============================] - 1s 198ms/step - loss: 0.2064 - accuracy: 0.9571 - val_loss: 6.7883 - val_accuracy: 0.7097
Epoch 5/6
6/6 [==============================] - 1s 205ms/step - loss: 0.6332 - accuracy: 0.9500 - val_loss: 7.9349 - val_accuracy: 0.7097
Epoch 6/6
6/6 [==============================] - 1s 202ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 5.0005 - val_accuracy: 0.7419


In [27]:
model5.save('inceptionV3_cropped.h5')

# Testing

## Cropped Images

In [25]:
#!pip install mediapipe

In [ ]:
import cv2
import mediapipe as mp
import math


class HandDetector:
    """
    Finds Hands using the mediapipe library. Exports the landmarks
    in pixel format. Adds extra functionalities like finding how
    many fingers are up or the distance between two fingers. Also
    provides bounding box info of the hand found.
    """

    def __init__(self, mode=False, maxHands=2, detectionCon=0.5, minTrackCon=0.5):
        """
        :param mode: In static mode, detection is done on each image: slower
        :param maxHands: Maximum number of hands to detect
        :param detectionCon: Minimum Detection Confidence Threshold
        :param minTrackCon: Minimum Tracking Confidence Threshold
        """
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.minTrackCon = minTrackCon

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(static_image_mode=self.mode, max_num_hands=self.maxHands,
                                        min_detection_confidence=self.detectionCon,
                                        min_tracking_confidence=self.minTrackCon)
        self.mpDraw = mp.solutions.drawing_utils
        self.tipIds = [4, 8, 12, 16, 20]
        self.fingers = []
        self.lmList = []

    def findHands(self, img, draw=True, flipType=True):
        """
        Finds hands in a BGR image.
        :param img: Image to find the hands in.
        :param draw: Flag to draw the output on the image.
        :return: Image with or without drawings
        """
        #blank_image = np.zeros((1080,1080,3), np.uint8)
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        allHands = []
        h, w, c = img.shape
        if self.results.multi_hand_landmarks:
            for handType, handLms in zip(self.results.multi_handedness, self.results.multi_hand_landmarks):
                myHand = {}
                ## lmList
                mylmList = []
                xList = []
                yList = []
                for id, lm in enumerate(handLms.landmark):
                    px, py, pz = int(lm.x * w), int(lm.y * h), int(lm.z * w)
                    mylmList.append([px, py, pz])
                    xList.append(px)
                    yList.append(py)

                ## bbox
                xmin, xmax = min(xList), max(xList)
                ymin, ymax = min(yList), max(yList)
                boxW, boxH = xmax - xmin, ymax - ymin
                bbox = xmin, ymin, boxW, boxH
                cx, cy = bbox[0] + (bbox[2] // 2), \
                         bbox[1] + (bbox[3] // 2)

                myHand["lmList"] = mylmList
                myHand["bbox"] = bbox
                myHand["center"] = (cx, cy)

                if flipType:
                    if handType.classification[0].label == "Right":
                        myHand["type"] = "Left"
                    else:
                        myHand["type"] = "Right"
                else:
                    myHand["type"] = handType.classification[0].label
                allHands.append(myHand)

                ## draw
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms,
                                               self.mpHands.HAND_CONNECTIONS)
#                     cv2.rectangle(img, (bbox[0] - 20, bbox[1] - 20),
#                                   (bbox[0] + bbox[2] + 20, bbox[1] + bbox[3] + 20),
#                                   (255, 0, 255), 2)
#                     cv2.putText(img, myHand["type"], (bbox[0] - 30, bbox[1] - 30), cv2.FONT_HERSHEY_PLAIN,
#                                 2, (255, 0, 255), 2)
        if draw:
            return allHands,img
        else:
            return allHands

    def fingersUp(self, myHand):
        """
        Finds how many fingers are open and returns in a list.
        Considers left and right hands separately
        :return: List of which fingers are up
        """
        myHandType = myHand["type"]
        myLmList = myHand["lmList"]
        if self.results.multi_hand_landmarks:
            fingers = []
            # Thumb
            if myHandType == "Right":
                if myLmList[self.tipIds[0]][0] > myLmList[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)
            else:
                if myLmList[self.tipIds[0]][0] < myLmList[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)

            # 4 Fingers
            for id in range(1, 5):
                if myLmList[self.tipIds[id]][1] < myLmList[self.tipIds[id] - 2][1]:
                    fingers.append(1)
                else:
                    fingers.append(0)
        return fingers

    def findDistance(self, p1, p2, img=None):
        """
        Find the distance between two landmarks based on their
        index numbers.
        :param p1: Point1
        :param p2: Point2
        :param img: Image to draw on.
        :param draw: Flag to draw the output on the image.
        :return: Distance between the points
                 Image with output drawn
                 Line information
        """

        x1, y1 = p1
        x2, y2 = p2
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        length = math.hypot(x2 - x1, y2 - y1)
        info = (x1, y1, x2, y2, cx, cy)
        if img is not None:
            cv2.circle(img, (x1, y1), 15, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), 15, (255, 0, 255), cv2.FILLED)
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
            cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
            return length, info, img
        else:
            return length, info

In [ ]:
detector = HandDetector(maxHands=1)
offset = 20
imgSize = IMAGE_HEIGHT_WIDTH
out_predicitons = []

mydir2 = "/content/drive/MyDrive/Projects/GP Project/Dataset/croppedDataset/testSet"
for i in os.listdir(mydir2):
  img = cv2.imread(mydir2 + "/" + i )
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  plt.imshow(img)

  #cv2.imshow("Image",img)

  #print(f"Count:{count} -  {folderPath}/{folder}/{image}")
  imgOutput = img.copy()
  hands, img = detector.findHands(img)
  #cv2.imshow(f"Image{count+1}",img)
  if hands:
      hand = hands[0]
      x, y, w, h = hand['bbox']
      imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
      imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]
      imgCropShape = imgCrop.shape
      aspectRatio = h / w
      if aspectRatio > 1:
          k = imgSize / h
          wCal = math.ceil(k * w)
          imgResize = cv2.resize(imgCrop, (wCal, imgSize))
          imgResizeShape = imgResize.shape
          wGap = math.ceil((imgSize - wCal) / 2)
          imgWhite[:, wGap:wCal + wGap] = imgResize


      else:
          k = imgSize / w
          hCal = math.ceil(k * h)
          imgResize = cv2.resize(imgCrop, (imgSize, hCal))
          imgResizeShape = imgResize.shape
          hGap = math.ceil((imgSize - hCal) / 2)
          imgWhite[hGap:hCal + hGap, :] = imgResize

      cv2.rectangle(imgOutput, (x - offset, y - offset-50),
                    (x - offset+90, y - offset-50+50), (255, 0, 255), cv2.FILLED)
      #Labels are stored in labels lis
      #cv2.putText(imgOutput, labels[index], (x, y -26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
      cv2.rectangle(imgOutput, (x-offset, y-offset),
                    (x + w+offset, y + h+offset), (255, 0, 255), 4)
      #       cv2.imshow("ImageCrop", imgCrop)
      #cv2.imshow("ImageWhite", imgWhite)
      imgWhite = cv2.cvtColor(imgWhite, cv2.COLOR_BGR2RGB)
      plt.imshow(imgWhite)
      plt.show()
      imgWhite = img_to_array(imgWhite)
      imgWhite = imgWhite.reshape((1, imgWhite.shape[0], imgWhite.shape[1], imgWhite.shape[2]))
      imgWhite = preprocess_input(imgWhite)
      print(imgWhite.shape)
      prediction = model.predict(imgWhite).argmax()
      out_predicitons.append(prediction)

      dic = list(train_dataset.class_indices)
      print(dic[prediction])
      
#cv2.waitKey(0)


## Regular Size Picture 

In [ ]:
my_images_input = []
my_images_labels = []


# getting the testing images output labels
if mode == "ORG":
  for images in glob("/content/drive/MyDrive/Projects/GP Project/Dataset/TestSet/*.png"):
      my_images_input.append(os.path.basename(images).split('.')[0].split('_')[0])
  for current_image in my_images_input:
    [my_images_labels.append(value) for key, value in my_dic.items() if current_image == key]
elif mode == "Crop":
  for images in glob("/content/drive/MyDrive/Projects/GP Project/Dataset/croppedDataset/testSet/*.png"):
      my_images_input.append(os.path.basename(images).split('.')[0].split('_')[0])
  for current_image in my_images_input:
    [my_images_labels.append(value) for key, value in my_dic.items() if current_image == key]
else:
  for images in glob("/content/drive/MyDrive/Projects/GP Project/Dataset/BgRemoved/Testing/*.png"):
    my_images_input.append(os.path.basename(images).split('.')[0].split('_')[0])
  for current_image in my_images_input:
    [my_images_labels.append(value) for key, value in my_dic.items() if current_image == key]

In [ ]:
if mode == "ORG":
  dir_path = "/content/drive/MyDrive/Projects/GP Project/Dataset/TestSet"
elif mode == "Crop":
  dir_path = "/content/drive/MyDrive/Projects/GP Project/Dataset/croppedDataset/testSet"
else: 
  dir_path = "/content/drive/MyDrive/Projects/GP Project/Dataset/BgRemoved/Testing"

out_predicitons = []
for i in os.listdir(dir_path):
    my_image = load_img(dir_path + '//' + i , target_size=(IMAGE_HEIGHT_WIDTH, IMAGE_HEIGHT_WIDTH))

    plt.imshow(my_image)
    plt.show()
    
    #preprocess the image
    my_image = img_to_array(my_image)
    my_image = my_image.reshape((1, my_image.shape[0], my_image.shape[1], my_image.shape[2]))
    my_image = preprocess_input(my_image)

    #make the prediction
    prediction = model.predict(my_image).argmax()
    out_predicitons.append(prediction)

    dic = list(train_dataset.class_indices)
    print(dic[prediction])

#print(confusion_matrix(my_images_labels, out_predicitons))
print(classification_report(my_images_labels, out_predicitons))


In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history['loss'], label='train loss')
plt.plot(hist.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# accuracies
plt.plot(hist.history['accuracy'], label='train acc')
plt.plot(hist.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')